# Visualization of Torsiondrive 2D scan result as energy countour and interactively showing molecular structures.


#### Known issues: this notebook doesn't work if opened in jupyterlab
### Install required packages for running this script (Run in terminal)
conda install -c conda-forge matplotlib plotly nglview ipywidgets jupyter-nbextension enable nglview --py --sys-prefix

### Specify path to scan.xyz (produced by torsiondrive 2D scan)

In [1]:
import urllib.request
# Use an example file downloaded from the torsiondrive_examples repo
url = "https://github.com/lpwgroup/torsiondrive_examples/raw/master/examples/propanol-2d/work_queue_qchem_geomeTRIC/scan.xyz"
# You can also try this one below for a "limited range" scan.
#url = "https://github.com/lpwgroup/torsiondrive_examples/raw/master/examples/range_limited_split/scan.xyz"
urllib.request.urlretrieve(url, 'scan.xyz')
# This path can be replaced by your own file path
scanxyz = "scan.xyz"

### Parse the scan.xyz file

In [2]:
import numpy as np
from torsiondrive.tools import read_scan_xyz
from torsiondrive.tools.plot_2d import format_2d_grid_data
# read contents of file into a dictionary
grid_data = read_scan_xyz(scanxyz)
# parse data into x_array, y_array and z_mat for plotting
x_array, y_array, z_mat = format_2d_grid_data(grid_data, verbose=True)
# convert abs energies to relative energies
z_mat = (z_mat - np.nanmin(z_mat)) * 627.509

grid_spacing: [15, 15]
grid_size:    [24, 24]


### Customized nglview class to visualize molecular structures

In [3]:
import nglview
from geometric.molecule import Molecule

class MyStructureTrajectory(nglview.Structure, nglview.Trajectory):
    """ 
    Custom nglview.Structure and nglview.Trajectory subclass
    For loading molecule files using geomeTRIC.molecule.Molecule

    Reference
    ---------
    http://nglviewer.org/nglview/latest/interface_classes.html
    """
    ext = "pdb"  # or gro, cif, mol2, sdf
    params = {}  # loading options passed to NGL
    id = '123121'
    def __init__(self, input_file, *args, **kwargs):
        mol = Molecule(input_file)
        # fill in required fields for PDB format
        if not mol.Data.get('resname'):
            mol.Data['resname'] = ['RES'] * mol.na
        if not mol.Data.get('resid'):
            mol.Data['resid'] = [1] * mol.na
        self.mol = mol
    
    def get_structure_string(self):
        return '\n'.join(self.mol.write_pdb(None))

    def get_coordinates(self, index):
        # return 2D numpy array, shape=(n_atoms, 3)
        return self.mol.xyzs[index]
    
    @property
    def n_frames(self):
        # return total frames
        return len(self.mol)

### Generate 2-D torsion angle vs energy contour

In [4]:
import plotly
import plotly.graph_objs as go

# The nglview widget for molecule
molview = nglview.NGLWidget(MyStructureTrajectory(scanxyz))

# plotly Contour plot
contour = go.Contour(x=y_array, y=x_array, z=z_mat, 
                     contours=dict(
                        # coloring ='heatmap', # this will smooth the colors
                        showlabels = True,
                        labelfont = dict(
                            family = 'Raleway',
                            size = 12,
                            color = 'white',
                        ),
                     ), ncontours=15, colorbar={'title':'Relative Energy (kcal/mol)', 'titleside': 'right'})
# Store the index of frames for each grid point
# because the grid_data dict should have the same order as the input file
frame_idx = {gid: idx for idx, gid in enumerate(grid_data)}
# click function
def show_structure_for_click(trace, points, selector):
    gid = (points.ys[0], points.xs[0])
    if gid in frame_idx:
        molview.frame = frame_idx[gid]
fig = go.FigureWidget([contour])
fig.layout = go.Layout(xaxis={'title': 'φ (degree)'}, yaxis={'title': 'ψ (degree)'}, width=700, height=700)
contour = fig.data[0]
contour.on_click(show_structure_for_click)

# display the plot and structure side by side
import ipywidgets
import IPython.display as display

topdownbox = ipywidgets.VBox([fig, molview])
## Finally, show.
display.display(topdownbox)

    'data': [{'colorbar': {'title': {'side': 'right', 'text': 'Relative Energy (…

## Alternatively, you can generate a pdf file of the energy contour

In [5]:
from torsiondrive.tools.plot_2d import plot_grid_contour
plot_filename = 'torsiondrive-2d-contour.pdf'
plot_grid_contour(grid_data, plot_filename, method='contourf')

In [6]:
plot_filename = 'torsiondrive-2d-heatmap.pdf'
plot_grid_contour(grid_data, plot_filename, method='imshow')